CephFS verwendet csi-cephfsplugin

Überprüfe, ob die CephFS-CSI-Plugins korrekt laufen:


In [ ]:
%%bash
sudo microk8s enable hostpath-storage

In [ ]:
%%bash
kubectl -n rook-ceph delete pod -l app=csi-cephfsplugin-provisioner


In [ ]:
%%bash
kubectl -n rook-ceph get pods -l app=csi-cephfsplugin

Zuerst brauchen wie ein Filesystem

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: ceph.rook.io/v1
kind: CephFilesystem
metadata:
  name: myfs
  namespace: rook-ceph
spec:
  metadataPool:
    replicated:
      size: 3
  dataPools:
    - replicated:
        size: 3
  preserveFilesystemOnDelete: true
  metadataServer:
    activeCount: 1
    activeStandby: true
EOF

Prüfen, ob CephFS korrekt eingerichtet ist
Überprüfe, ob CephFS bereits läuft:

In [ ]:
! kubectl -n rook-ceph get CephCluster,cephfilesystems


In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRole
metadata:
  name: ceph-csi-provisioner
rules:
  - apiGroups: [""]
    resources: ["persistentvolumes"]
    verbs: ["get", "list", "watch", "create", "delete", "patch", "update"]
  - apiGroups: ["storage.k8s.io"]
    resources: ["storageclasses"]
    verbs: ["get", "list", "watch"]
  - apiGroups: ["storage.k8s.io"]
    resources: ["volumeattachments"]
    verbs: ["get", "list", "watch", "patch", "update"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: ClusterRoleBinding
metadata:
  name: ceph-csi-provisioner-binding
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: ceph-csi-provisioner
subjects:
  - kind: ServiceAccount
    name: rook-csi-cephfs-provisioner
    namespace: rook-ceph
EOF


StorageClass für Rook-Ceph erstellen

In [ ]:
! kubectl -n rook-ceph delete pod -l app=csi-cephfsplugin-provisioner


In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: rook-cephfs
provisioner: rook-ceph.cephfs.csi.ceph.com
parameters:
  clusterID: rook-ceph
  fsName: myfs
  pool: myfs-data0
  csi.storage.k8s.io/provisioner-secret-name: rook-csi-cephfs-provisioner
  csi.storage.k8s.io/provisioner-secret-namespace: rook-ceph
  csi.storage.k8s.io/node-stage-secret-name: rook-csi-cephfs-node
  csi.storage.k8s.io/node-stage-secret-namespace: rook-ceph
  csi.storage.k8s.io/fstype: ext4
reclaimPolicy: Delete
EOF


Persistent Volume Claim (PVC) für Ceph RBD erstellen

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: ceph-pvc
spec:
  accessModes:
    - ReadWriteOnce
  resources:
    requests:
      storage: 5Gi
  storageClassName: rook-cephfs
EOF

Pod mit Ceph-Speicher starten

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: v1
kind: Pod
metadata:
  name: ceph-test-pod
spec:
  containers:
  - name: busybox
    image: busybox
    command: ["/bin/sh", "-c", "while true; do echo 'Ceph läuft!' >> /mnt/test/data.txt; sleep 10; done"]
    volumeMounts:
    - mountPath: "/mnt/test"
      name: ceph-storage
  volumes:
  - name: ceph-storage
    persistentVolumeClaim:
      claimName: ceph-pvc
EOF

In [ ]:
%%bash
kubectl describe pvc ceph-pvc


In [ ]:
%%bash
kubectl delete pod ceph-test-pod
kubectl delete sc rook-ceph-block
kubectl delete pvc ceph-pvc